In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import zipfile

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/match_prediction/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d rounakbanik/the-movies-dataset --force
# !kaggle datasets download -d excel4soccer/espn-soccer-data --force
zip_path = "the-movies-dataset.zip"
extract_path = "movies_data"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
  zip_ref.extractall(extract_path)

print("Extraction completed.")





Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
 98% 224M/228M [00:00<00:00, 709MB/s] 
100% 228M/228M [00:00<00:00, 744MB/s]
Extraction completed.


# Simple Recommenders system

In [3]:
import pandas as pd

metadata = pd.read_csv('movies_data/movies_metadata.csv')

/tmp/ipython-input-3751686374.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('movies_data/movies_metadata.csv')


In [4]:
metadata.shape

(45466, 24)

In [5]:
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


you must come up with a weighted rating that takes into account the average rating and the number of votes it has accumulated. Such a system will make sure that a movie with a 9 rating from 100,000 voters gets a (far) higher score than a movie with the same rating but a mere few hundred voters.

Since you are trying to build a clone of IMDB's Top 250, let's use its weighted rating formula as a metric/score. Mathematically, it is represented as follows:

\begin{equation} \text Weighted Rating (\bf WR) = \left({{\bf v} \over {\bf v} + {\bf m}} \cdot R\right) + \left({{\bf m} \over {\bf v} + {\bf m}} \cdot C\right) \end{equation}

In the above equation,

- v is the number of votes for the movie;

- m is the minimum votes required to be listed in the chart;

- R is the average rating of the movie;

- C is the mean vote across the whole report.

Determining an appropriate value for m is a hyperparameter that you can choose accordingly since there is no right value for m. You can consider it as a preliminary negative filter that will simply remove the movies which have a number of votes less than a certain threshold m. The selectivity of your filter is up to your discretion.

In this tutorial, you will use cutoff m as the 90th percentile. In other words, for a movie to be featured in the charts, it must have more votes than at least 90% of the movies on the list. (On the other hand, if you had chosen the 75th percentile, you would have considered the top 25% of the movies in terms of the number of votes garnered. As percentile decreases, the number of movies considered will increase).

In [6]:
# Calculate the mean rating across all movies (c)
# On a scale of 10
c = metadata["vote_average"].mean()

print(c)

5.618207215134185


In [7]:
# Calculate the minimum number of votes (m) by a movie in the 90th percentile

m = metadata["vote_count"].quantile(0.90)
print(m)

160.0


In [8]:
# Filter out all Qulaified movies into a new Data Frame

q_movies = metadata.copy().loc[metadata["vote_count"]>=m]

q_movies.shape

(4555, 24)

In [9]:
metadata.shape

(45466, 24)

From the above output, it is clear that there are around 10% movies with vote count more than 160 and qualify to be on this list.

In [10]:
# Function that computes the wighted rating of each movie
def weighted_rating(x, m=m, c=c):
  v = x["vote_count"]
  R = x["vote_average"]

  # Calculate based on the IMDB formula
  return (v/(v+m) * R) + (m/(m+v) * c)

In [11]:
# Define a new feature "Score" and claculte its value with "weighted_ratings"

q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [12]:
# Sort movies based on score calculated above

q_movies = q_movies.sort_values("score", ascending=False)

In [13]:
# Print the top 15 movies

q_movies[["title", "vote_count", "vote_average", "score"]].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


# Content-Based Recommender